In [1]:
%%capture
%load_ext jupyter_probcomp.magics
%matplotlib inline

In [2]:
import itertools
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Load the read infer predict layer (ripl) and set venture's random seed to 1 to
ensure reproducibility.

In [3]:
%ripl --seed 1

Set seed of a new VentureScript RIPL to 1.00.


In [4]:
%%venturescript
// MODEL
assume cause_prior = 0.01;

assume disease_1 ~ flip(cause_prior) #symptom_1:0;
assume disease_2 ~ flip(cause_prior) #symptom_1:1;
assume disease_3 ~ flip(cause_prior) #symptom_1:2 #symptom_2:0;

assume disease_4 ~ flip(cause_prior) #symptom_2:1;
assume disease_5 ~ flip(cause_prior) #symptom_2:2;
assume disease_6 ~ flip(cause_prior) #symptom_2:3;

assume get_causal_link = (parent) -> {
    if (parent) {
        0.01 // Probability of a leak
    } else {
        1.   
    }
};

assume get_effect = (parents, effect) -> {
    if (size(parents) == 0) {
        effect
    } else {
        get_effect(rest(parents), effect * get_causal_link(first(parents)))
    }
};

assume noisy_or = (parents) ~> {
    p_spontaneous = 0.001;
    flip(1 - ((1-p_spontaneous) * get_effect(to_list(parents), 1)))
};
// Helper function for evaluation
define get_diseases =  () -> {sample([
    disease_1,
    disease_2,
    disease_3,
    disease_4,
    disease_5,
    disease_6,
])};

In [5]:
%%venturescript
log_joint_at(default, all)

[-0.060302015121008655]

In [6]:
%%venturescript
// OBSERVATIONS
observe noisy_or([disease_1, disease_2, disease_3]) = True;
observe noisy_or([disease_3, disease_4, disease_5, disease_6]) = True;

In [7]:
%%venturescript
// INFERENCE
define resimulation_mh  = () -> {
    mh(default, all, 1)
};

define single_site_mh  = () -> {
    mh(default, one, 1)
};

define single_site_gibbs = () -> {
    gibbs(default, one, 1);
};

define global_gibbs = () -> {
    gibbs(default, all, 1);
};

define block_gibbs = () -> {
    gibbs(quote(symptom_1), all, 1);
    gibbs(quote(symptom_2), all, 1)
};
